# Introduction

[Bert Carremans 노트북](https://www.kaggle.com/bertcarremans/data-preparation-exploration/data#Introduction) 필사

**KEY POINTS**
* EDA(Exploratory Data Analysis)
* Feature Engineering(Dummification, Interaction)
* Feature Selection(Zero and Low Variance 제거, SelectFromModel) 

# Section
1. [Visual inspection of your data](#1)
2. **[Defining the metadata](#2)**
3. [Descriptive statistics](#3)
4. **[Handling imbalanced classes](#4)**
5. [Data quality checks](#5)
6. [Exploratory data visualization](#6)
7. [Feature engineering](#7)
8. **[Feature selection](#8)**
9. [Feature scaling](#9)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a name='1'></a>
# 1. Visual inspection of your data
## 1.1 Data Load

In [ ]:
train = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/train.csv')
test = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/test.csv')

## 1.2 Data at first sight

## 데이터 분석 요약
* [train set < test set](#1)
* 비슷한 그룹에 속하는 feature에는 이름에 태그가 지정 (ind, reg, car, calc)
* feature이름에는 binary와 category를 나타내는 bin, cat이 postfix로 포함
* -1 값은 Null을 의미
* target은 보험 청구를 한다(=1), 보험 청구를 하지 않는다(=0)인 binary 데이터, 0이 압도적으로 많음 



In [ ]:
train.head()  # defalut number = 5

In [ ]:
train.tail()

중복되는 row가 존재하는지 확인

In [ ]:
train.duplicated()

In [ ]:
train.shape

In [ ]:
test.shape

test set은 target 변수가 빠져있기에 변수가 1개 줄어들음


14개의 categorical 변수에 대한 dummy 변수(이진화)를 만들 수 있다. // 타이타닉에서 카테고리를 수치화 시킨 것과 같이

In [ ]:
train.info()

<a name='2'></a>
# 2. Defining the metadata

* 본 커널의 특징은 데이터 관리를 위해 Meta Data를 만들어 활용했다는 것
> Meta Data란? 
  features에 대한 정보라고 이해하자

어떤 조건의 feature만 사용하고 싶을 때 그때마다 코드를 작성하기 번거롭다. 

편리성을 위해 Meta Data를 미리 만들어 놓고 간단한 코드로 feature engineering!(분석, 시각화, 모델링..)


**META DATA 구성**
* role: input, ID, target 
* level: nominal, interval, ordinal, binary - [Feature type 참고자료](http://blog.heartcount.io/dd)
* keep: True or False - 데이터 활용 여부
* dtype: int, float, str

In [ ]:
data = []
for f in train.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # Defining the data type 
    dtype = train[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [ ]:
meta

In [ ]:
# 사용할 nominal 타입을 불러올 때
meta[(meta.level == 'nominal') & (meta.keep)].index  

In [ ]:
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

<a name='3'></a>
# 3. Descriptive statistics

* meta data 덕분에 계산하고자 하는 변수를 쉽게 선택 가능
* describe 메소드 사용하여 데이터의 통계 확인
* 범주형 범수에는 의미가 없으니 실수형 변수에 사용하여 평균, 표준편차 등을 알수 있음

### 1. Interval 변수


In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

* reg 변수들중에는 ps_reg_03에만 -1(Null data)가 있다.
* car 변수들중에는 ps_car_12, ps_car_14에 -1(Null data)가 있다.
* calc 변수들에는 -1(NUll data)는 없다.
* 변수별로 min과 max의 range가 다르기에, 스케일링을 적용 필요
* interval 변수들의 범위는 그렇게 크지 않음.

### 2. Ordinal 변수

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

* ps_car_11 변수에만 -1(Null data)가 있다.
* 모두 min, max range가 다르므로 scaling을 진행 필요

### 3. Binary variables

In [ ]:
v = meta[(meta.level == 'binary') & (meta.keep == True)].index
train[v].describe()

Train 데이터에서 target은 3.645% 
대부분의 값이 0으로 되어있는 **strongly imbalanced**

<a name='4'></a>
# 4. Handling imbalanced classes

Target = 1인 Record의 비율이 너무 적다. 즉, 모두다 target을 0으로 예측해도 얼마안되는 1만 틀린것으로 파악됨
[해결방법](https://dining-developer.tistory.com/27)들 중 **Under sampling** 선택

**UnderSampling**은 많은 부분을 차지하는 세트에 적은 부분을 차지하는 세트 수준으로 감소시키는 방법이다.
물론, overfitting 문제를 해결할 수는 있지만, 중요한 데이터가 삭제될 위험이 있다.

In [ ]:
desired_apriori=0.10

# Get the indices per target value
idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index

# Get original number of records per target value
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# Calculate the undersampling rate and resulting number of records with target=0
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('Rate to undersample records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# Randomly select records with target=0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# Construct list with remaining indices
idx_list = list(undersampled_idx) + list(idx_1)

# Return undersample data frame
train = train.loc[idx_list].reset_index(drop=True)
under_rate = train['target'].sum() / train['target'].count()
print(f'Under sampling으로 변환된 target의 비율 : {under_rate} %')

* undersampling_rate: target=0이 몇%가 되어야 target=1이 전체 데이터에서 desired_apriori가 되는지의 대한 비율

* desired_apriori = 0.10 는 undersampling 후 나오게될 target = 1의 비율

<a name='5'></a>
# 5. Data Quality Checks
## 5.1 Checking missing values

In [ ]:
vars_with_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings / train.shape[0]
        
        print(f'Variable {f} has {missings} records {missings_perc:.2%} with missing values')
print(f'In total, there are {len(vars_with_missing)} varialbles with missing values')

* Missing Values(Null Data)인 -1을 각 변수별로 찾아서, 비율을 확인한것 
* 생각보다 Missing Values가 많은 변수가 있다. ps_res_03, ps_car_03_cat, ps_car_05_cat …
* 총 12개의 변수에서 Missing values 존재

## SimpleImputer
SimpleImputer?
: 결치 값을 채우기 위한 클래스이다.

SimpleImputer 사용방법
* SimpleImputer constructor 호출한다.
* missing_values: 어떤 값이 결치 값인지, default은 NaN이다.
* strategy: 'mean'은 평균값, 'most_frequent' 최빈 값이다.
* 리턴값을 col=fit_transform(col).ravel() 호출한다.

In [ ]:
# Dropping the variables with too many missing values
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace=True, axis=1)
meta.loc[(vars_to_drop),'keep'] = False  # Updating the meta

# Imputing with the mean or mode
mean_imp = SimpleImputer(missing_values=-1, strategy='mean')
mode_imp = SimpleImputer(missing_values=-1, strategy='most_frequent')
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel() 
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel() # ordinal value


* Missing value이있는 다른범주형 변수의 경우 Missing value -1을 그대로 둠
* ps_reg_03 (continuous)의 18%의 Missing value는 평균으로 바꿉니다.
* ps_car_11 (ordinal)의 1개의 Misisng values는 최빈값으로 바꿉니다.
* ps_car_12 (continuous)의 단 1개의 Missing value 평균으로 바꿉니다.
* ps_car_14 (continuous)의 7% Missing values는 평균으로 바꿉니다.

## 5.2 Checking the cardinality of the categorical variables
 > cardinality: 변수에있는 서로 다른 값의 수

In [ ]:
v = meta[(meta['level'] == 'nominal') & (meta['keep'])].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print(f'Variable {f} has {dist_values} distinct values')

* nominal 변수에서 더미 변수를 만들 것이므로 고유 한 값이 많은 변수가 있는지 확인해야함. 이러한 변수는 많은 더미 변수를 생성
* ps_car_11_cat이 104개의 distinct data를 가집니다.

In [ ]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
train_encoded, test_encoded = target_encode(train['ps_car_11_cat'],
                                            test['ps_car_11_cat'],
                                            target=train.target,
                                            min_samples_leaf=100,
                                            smoothing=10,
                                            noise_level=0.01)

train['ps_car_11_cat_te'] = train_encoded
train.drop('ps_car_11_cat', axis=1, inplace=True)
meta.loc['ps_car_11_cat', 'keep'] = False  # Updating the meta
test['ps_car_11_cat_te'] = test_encoded
test.drop('ps_car_11_cat', axis=1, inplace=True)

ps_car_11_cat 처리
by using target_encoder

categorical 변수는 target encoder를 사용하여 renaming

In [ ]:
train_encoded

In [ ]:
v = meta[(meta['level'] == 'nominal') & (meta['keep'])].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print(f'Variable {f} has {dist_values} distinct values')

<a name='6'></a>
# 6. Exploratory Data Visualization

### MetatData를 이용한 Visualization

## 6.1 Categorical variables

다음과 같이 v에 동일한 특성(categorical)을 갖는 columns을 for 문을 이용해 데이터 시각화를 진행한다. 이렇게 되면 같은 특성인 경우에 같은 그래프를 통해 분석할 수 있는 통일성을 가질 수 있다.

### barplot: 막대그래프 matplotlib 보다 색감 있는 그래프를 만들어준다
[seaborn 참고이미지](https://chuyinchule.tistory.com/16)

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    plt.figure()
    fig, ax = plt.subplots(figsize = (10,5))
    
    # Calculate the percentage of target = 1 per category value
    cat_perc = train[[f, 'target']].groupby([f], as_index = False).mean()
    cat_perc.sort_values(by = 'target', ascending = False, inplace = True)
    
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax = ax, x = f, y = 'target', data = cat_perc, order= cat_perc[f])
    plt.ylabel('% Target', fontsize = 18)
    plt.xlabel(f, fontsize = 18)
    plt.tick_params(axis = 'both', which = 'major', labelsize = 18)
    plt.show()

Categorical variables와 Target = 1인 고객 비율을 살펴보자.

Missing value가 있는 변수에서 알 수 있듯이 
Missing valued가 차지하는 범위가 크기에
Missing value를 다른 값으로 대체하는 대신 별도의 범주 값으로 유지하는 것이 좋다.

Missing value가 있는 고객은 보험 청구를 요청할 가능성이 훨씬 더 높은 것으로 보인다.

## 6.2 Interval variables 

In [ ]:
def corr_heatmap(v):
    correlations = train[v].corr()

    # Create color map ranging between two colors
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75})
    plt.show();
    
v = meta[(meta.level == 'interval') & (meta.keep)].index
corr_heatmap(v)

Interval variables 간의 상관 관계를 확인합니다.
heatmap은 변수 간의 상관 관계를 시각화하는 좋은 방법입니다.
아래의 변수들은 강한 상관 관계를 가집니다.
* ps_reg_02 & ps_reg_03 (0.7)
* ps_car_12 & ps_car13 (0.67)
* ps_car_12 & ps_car14 (0.58)
* ps_car_13 & ps_car15 (0.67)

Seaborn은 변수들 사이의 (선형) 관계를 시각화할 수 있는 몇 가지 유용한 플롯을 가지고 있다. 우리는 변수들 사이의 관계를 시각화하기 위해 Pairplot 사용할 수 있습니다.
하지만 Heatmap에서 이미 제한된 수의 상관 변수를 보여 주었기 때문에, 우리는 각각의 높은 상관 관계를 가진 변수들을 개별적으로 살펴보도록 하겠습니다.

### lmplot
>lmplot은 x와 y의 상관관계를 파악하고 싶을 때, 이를 사용한다. 
 특히 여기서는 둘 feature간의 상관관계를 나타내기 위해서 사용한다. scatter_kws는 점의 크기를 설정하고 palette는 색을 정의한다.

x，y ： x축 ,y축 이름 
data : 데이터 프레임
fit_ref : 회귀선그리기
hue : 항목별로 구별
{scatter, line} _kws : 점의 속성 변경
markers : 점의 모양 설정


In [ ]:
# 속도를 높이기 위해 학습 데이터의 일부를 가져옵니다.
s = train.sample(frac = 0.1)

In [ ]:
sns.lmplot(x='ps_reg_02', y='ps_reg_03', data=s, hue='target',
           palette='Set1', scatter_kws={'alpha': 0.3})
plt.show()

ps_reg_02 및 ps_reg_03 regression line에서 알 수 있듯이 두 변수 사이에는 linear relationship이 있습니다.
hue 매개 변수는 target = 0과 target = 1에 대한 regression line이 동일함을 알 수 있습니다.

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_13', data=s, hue='target',
           palette='Set1', scatter_kws={'alpha': 0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_14', data=s, hue='target',
           palette='Set1', scatter_kws={'alpha': 0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_15', y='ps_car_13', data=s, hue='target',
           palette='Set1', scatter_kws={'alpha': 0.3})
plt.show()

**correlated variables**를 통해 PCA를 수행하여 차원을 줄일 수 있습니다.

하지만 correlated variables의 수가 적기 때문에 모델이 heavy-lifting처럼 무겁게 수행됩니다.

## 6.3 Checking the correlations betwwen ordinal variables

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
corr_heatmap(v)

Ordinal 변수에서는 뚜렷한 correlations을 볼 수 없지만 

targeting value로 그룹화하면 어떻게 값이 분포되는지 알 수 있습니다.

<a name='7'></a>
# 7. Feature engineering

## 7.1 Creating dummy variables

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
print(f'Before dummification we have {train.shape[1]} variables in train.')
train = pd.get_dummies(train, columns= v, drop_first= True)
print(f'After dummification we have {train.shape[1]} variables in train.')

Categorical variables의 값은 순서나 크기를 나타내지 않음. 예를들어, group"2" != group"1" * 2
--> Categorical변수를 다룰 더미 변수를 만들 수 있다.

[dummy 변수란?](https://kkokkilkon.tistory.com/37)
총 52(109-57)개의 dummy 변수를 생성하였다.

## 7.2 Creating interaction variables

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
poly = PolynomialFeatures(degree = 2, interaction_only= False, include_bias= False)
interactions = pd.DataFrame(data = poly.fit_transform(train[v]), columns=poly.get_feature_names(v))
interactions.drop(v, axis = 1, inplace = True) # Remove the original columns

# Concat the interaction variables to the train data
print(f'Before creating interactions we have {train.shape[1]} variables in train.')

train = pd.concat([train, interactions], axis = 1)

print(f'After creating interactions we have {train.shape[1]} variables in train.')

get_feature_names 메서드를 사용하여 interactions variables을 추가한다.

<a name='8'></a>
# 8. Feature selection
## 8.1 Removing features with low or zero variance

In [ ]:
selector = VarianceThreshold(threshold=0.01)
selector.fit(train.drop(['id', 'target'], axis = 1)) # Fit to train without id and target variables

f = np.vectorize(lambda x : not x) # Function to toggle boolean_array elements
v = train.drop(['id', 'target'], axis = 1).columns[f(selector.get_support())]
print(f'{len(v)} variables have too low variance.')
print(f'These variables are {list(v)}')

* 변동이 없거나 매우 낮은 특성을 제거하는 것이다.  **분산이 매우 작은 feature 삭제**
* Sklearn에는 VarianceThreshold라는 편리한 방법이 있다. 기본적으로 분산이 0 인 기능을 제거한다.
* 이전 단계에서 0 분산 변수가 없음을 확인 했으므로이 대회에는 적용되지 않습니다.
* 그러나 분산이 1 % 미만인 특성을 제거하면 31 개의 변수가 제거됩니다.
* 분산을 기반으로 선택하면 다소 많은 변수(31개)를 잃게됩니다. 그러나 변수가 너무 많지 않기 때문에 classifier가 선택하도록 할 것입니다.
* 더 많은 변수가있는 데이터 세트의 경우 처리 시간을 줄일 수 있습니다.
* Sklearn은 또한 다른 기능 선택 방법과 함께 제공됩니다.
* 이러한 메서드 중 하나는 another classifier가 최상의 기능을 선택하고 계속 진행하도록하는 SelectFromModel입니다.
* 아래에서는 Random Forest로 수행하겠습니다.

## 8.2 Selecting features with a Random Forest and SelectFromModel

RandomForest에서 선택한 Feature의 중요성을 파악하고 

SelectFromModel에서 RandomForest에서 고른 Feature들의 평균값 보다 큰 feature들을 선택한다.

In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']

feat_labels = X_train.columns

rf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)

rf.fit(X_train, y_train)
importances = rf.feature_importances_

indices = np.argsort(rf.feature_importances_)[::-1]
# np.argsort 함수는 배열을 오름차순으로 정렬할 수 있는 인덱스들을 반환한다. 배열 [3, 1, 2]에 argsort 함수를 적용하면 [1, 2, 0]을 반환한다. 
# [::-1]에 의해 내림차순으로 정렬할 수 있는 인덱스가 된다.

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]], importances[indices[f]]))

여기서는 랜덤 포레스트의 feature importances를 기준으로 기능 선택을 할 것입니다.
Sklearn의 SelectFromModel을 사용하면 유지할 변수 수를 지정할 수 있습니다.
feature importances 수준에 대한 threshold를 수동으로 설정할 수 있습니다.
그러나 우리는 단순히 상위 50 % 최고의 변수를 선택합니다.
위의 코드는 Sebastian Raschka의 GitHub 저장소에서 가져 왔습니다.

In [ ]:
sfm = SelectFromModel(rf, threshold='median', prefit=True)
print(f'Number of features before selection : {X_train.shape[1]}')

n_features = sfm.transform(X_train).shape[1]
print(f'Number of features after selection : {n_features}')
selected_vars = list(feat_labels[sfm.get_support()])

SelectFromModel을 사용하여 사용할 prefit classifier와 feature importances에 대한 threshold을 지정할 수 있습니다.

get_support 메소드를 사용하면 train 데이터의 변수 수를 제한 할 수 있습니다.

In [ ]:
# train 데이터에 target까지 추가
train = train[selected_vars + ['target']]

<a name='9'></a>
# 9. Feature scaling

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(train.drop(['target'], axis = 1))

train 데이터에 standardscaler를 적용 할 수 있습니다.

이 작업이 완료되면 일부 classifier가 더 잘 작동됩니다.